In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing as pp
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/user_city_sim.csv", index_col=0)
df

In [ ]:
df2 = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/user_city_dummy.csv", index_col=0)
df2

In [ ]:
df3 = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/user_city_rank.csv", index_col=0)
df3

In [ ]:
new_user = df3.loc[(df3.index == 11)]



In [ ]:
df4 = df3.drop(df3.loc[(df3.index == 11)].index)

In [ ]:
def df_no_words(df):
    df.reset_index(inplace=True)
    idk = df.drop(columns=['city'])
    idk.set_index(['user', 'label_id'], inplace=True)
    idk.rename(columns={'rank': 'top'}, inplace=True)
    
    return idk


In [ ]:
ready = df_no_words(df4)

In [ ]:
ready

In [ ]:
import random
index2 = random.sample(range(152), 30)

In [ ]:
index2[0]

In [ ]:
ready.xs(index2[0], level='user', drop_level=False)

In [ ]:
test_set = pd.DataFrame()
for i in range(len(index2)):
    test_set = test_set.append(ready.xs(index2[i], level='user', drop_level=False))


In [ ]:
test_set

In [ ]:
train_set = ready.copy()

In [ ]:
train_set.reset_index(level=1, inplace=True)

In [ ]:
for i in range(len(index2)):
    train_set = train_set.drop(train_set.loc[(train_set.index == index2[i])].index)

In [ ]:
ready

In [ ]:
normalized = pp.normalize(idk2)
normalized = pd.DataFrame(normalized, index=idk2.index, columns=idk2.columns)
normalized

In [ ]:
from sklearn.preprocessing import StandardScaler
scaled_features = StandardScaler().fit_transform(idk2.values)

scaled_features_df = pd.DataFrame(scaled_features, index=idk2.index, columns=idk2.columns)


In [ ]:
sm = SMOTE(random_state = 33)
x_train, X_train_new, y_train, y_test = train_test_split(ready.drop('top', axis=1),
                                                    ready.top, test_size= 0.2, random_state=12)


In [ ]:

X_train_new, y_train_new = sm.fit_sample(x_train, y_train.ravel())

In [ ]:
x_train

In [ ]:
train_set['top']

In [ ]:

y_train_set = train_set['top']
train_set.drop(columns=['top'], inplace=True)


In [ ]:
x_test_set = test_set['top']
#test_set.drop(columns=['top'], inplace=True)

In [ ]:
nr = NearMiss()
X_train_NM, y_train_NM = nr.fit_sample(x_train, y_train)

In [ ]:
scaled_features_df

In [ ]:
x_train

### Logistic Regression

Ranks lbfgs  
Accuary is .97
But precision and recall, f1-score for 1 = 0

#### SMOTE lbfgs
P: .07  
R= .27  
f1=  .11

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs', max_iter=10000).fit(X_train_new, y_train_new)
predictions_lr = clf.predict(x_test)

In [ ]:
print(classification_report(y_test, predictions_lr))
print(confusion_matrix(y_test,predictions_lr))


### KNN


#### Rank
precision: .12  
recall: .01  
f1: .01  

#### SMOTE
P:  .08  
R:  .28  
f1: .12  

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier()
neigh.fit(X_train_new, y_train_new)
predictions_kn = neigh.predict(x_test)

In [ ]:
print(classification_report(y_test, predictions_kn))
print(confusion_matrix(y_test,predictions_kn))


### SVM


#### Rank
0 for all - rbf

#### SMOTE
rbf
P= .09  
R= .64  
f1 = .15  


In [ ]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='rbf')
svclassifier.fit(X_train_new, y_train_new)
predictions_svm = svclassifier.predict(x_test)


In [ ]:
print(confusion_matrix(y_test,predictions_svm))
print(classification_report(y_test,predictions_svm))

In [ ]:
test.top.value_counts()

### Decision Tree


#### rank
precision: .25  
recall: .26  
f1: .25  


#### SMOTE
P= .20  
R= 26  
F1= .22  

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train_new, y_train_new)
predictions_dt = clf.predict(x_test)

In [ ]:
print(classification_report(y_test, predictions_dt))
print(confusion_matrix(y_test,predictions_dt))


### Random Forest


#### Rank
Precision: .94  
recall: .10  
f1:  .19


#### SMOTE
P= .83  
R= .13  
F1= .23  

In [ ]:
#'max_depth': 25, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100} random_state=12

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=12, max_depth=25)
clf.fit(train_set, y_train_set)
predictions_rf = clf.predict(x_test)
probs_rf = clf.predict_proba(x_test)

In [ ]:
y_train_set

In [ ]:
clf.get_params()

In [ ]:
print(classification_report(y_test, predictions_rf))
print(confusion_matrix(y_test,predictions_rf))


In [ ]:
rf_tts = RandomForestClassifier(random_state=12, max_depth=25)
rf_tts.fit(X_train_new, y_train_new)
predictions_rf_tts = rf_tts.predict(x_test)
probs_rf_tts = rf_tts.predict_proba(x_test)

In [ ]:
print(classification_report(y_test, predictions_rf_tts))
print(confusion_matrix(y_test,predictions_rf_tts))

In [ ]:
x_test.sort_values('user')

In [ ]:
x_test

In [ ]:

user_ready = df_no_words(new_user)

In [ ]:
user_ready.drop(columns='top', inplace=True)

In [ ]:
user_ready

In [ ]:
one_predict = clf.predict(user_ready)
one_probs = clf.predict_proba(user_ready)

In [ ]:
one_predict2 = rf_tts.predict(user_ready)
one_probs2 = rf_tts.predict_proba(user_ready)

In [ ]:
(one_predict == 1).sum()

In [ ]:
one_probs

In [ ]:
def split_array(a, df):
    lst1 = list(a[:, 0])
    lst2 = list(a[:, 1])
    df['prob1_group'] = pd.Series(lst1, index=df.index)
    df['prob2_group'] = pd.Series(lst2, index=df.index)
    
    return df

In [ ]:
user22 = split_array(one_probs, new_user)
user22

In [ ]:
sorted_user_group = user2.sort_values('prob2_group', ascending=False)
sorted_user = user2.sort_values('prob2', ascending=False)

In [ ]:
sorted_user.head(10)

In [ ]:
sorted_user_group.head(10)

In [ ]:
sorted_user[sorted_user['rank'] == 1]

In [ ]:
unine

In [ ]:
predictions_rf

In [ ]:
y_test

In [ ]:
probs_rf

In [ ]:
lst1 = list(probs_rf[:, 0])

In [ ]:
lst2 = list(probs_rf[:, 1])

In [ ]:
probs_rf[:, 1]

In [ ]:
df_y_test = pd.DataFrame(y_test)
df_y_test

In [ ]:
df_y_test['prob_new'] = pd.Series(prob, index=df_y_test.index)

In [ ]:
df_y_test['prob_1'] = pd.Series(lst1, index=df_y_test.index)
df_y_test['prob_2'] = pd.Series(lst2, index=df_y_test.index)

In [ ]:
df_y_test.sort_values('prob_2', ascending=False)

In [ ]:
df_y_test[df_y_test['prob_2'] > .47]

In [ ]:
predict_lst = list(predictions_rf)
df_y_test['predict'] = pd.Series(predict_lst, index=df_y_test.index)


In [ ]:
L = list(data)
A = list(data)
data_frame = pd.DataFrame(L) 
data_frame['A'] = pd.Series(A, index=data_frame.index)

In [ ]:
n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(clf, hyperF, cv = 2, verbose = 3, 
                      n_jobs = -1)
bestF = gridF.fit(X_train_new, y_train_new)

In [ ]:
predictions_rfcv = bestF.predict(x_test)

In [ ]:
print(classification_report(y_test, predictions_rfcv))
print(confusion_matrix(y_test,predictions_rf))


bestF.best_params_

{'max_depth': 25,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}
 random_state=12
 
 
 P= .66  
 R= .13  
 F1 = .22  
 
 
 
 4074  4  
 126   19  

In [ ]:
clf.get_params()